# 🚖 Taximeter - Python Program

This Python program simulates a **taximeter**, calculating fares based on time and demand conditions. It includes logging, configurable rates, and unit tests to ensure reliability.

### Features  
-  **Logging System**: Keeps track of ride history for traceability.  
-  **Unit Testing**: Ensures the program runs correctly.  
-  **Ride History**: Stores past rides in a text file (`ride_logs.json`).  
-  **Configurable Pricing**: Prices adapt based on demand.  

### Fare Calculation  
-  **Stationary**: 0.02€/s  
-  **Motioning**: 0.05€/s  
-  **High Demand**: +10% increase  
-  **Low Demand**: -10% discount  

### How It Works  
1.  **Start a Ride**: Press `Enter`.  
2. ⏸️ **Stop the Taxi**: Type `stop`.  
3. ⏯️ **Resume Moving**: Type `go`.  
4. ⏹️ **End the Ride**: Type `end`.  
5. ❌ **Exit the Program**: Press `Escape`.  

### Demand Levels  
- 🔴 **High Demand**: Extra charge applied during peak hours.  
- 🟡 **Normal Demand**: Standard rate used.  
- 🟢 **Low Demand**: Discount applied during off-peak hours.  

### Logging & Configuration  
- **Ride logs**: Stored in `ride_logs.json`.  
- **Custom pricing**: Modify `rates.json` to adjust fares.  
- **Run tests**: Use `pytest` for unit testing.  


In [10]:
import time
import datetime
import json
import os

In [11]:
def show_welcome():
    print("\n--- Welcome to your taximeter!  🚕 ---")
    print("""This program calculates the fare in km of a ride in euros.
            \n let me show you how we work first 😉

            The rates used are as follows:

            ⏸️ Taxi stopped: 2 cents per second.
            ▶️ Taxi in motion: 5 cents per second.
            ⬆️ High demand level: extra 10%.
            ⬇️ Low demand level: minus 10%.""")

    print("""\nCommands are:\n

            Next when you are on a ride you must use the follow accordingly:\n

            ▶️ Start:  Press 'Enter' to begin a trip.\n
            ⏸️ Stop:   Type 'stop' when the taxi is stationary.\n
            ⏯️ Go:     Type 'go' when the taxi is in motion.\n
            ⏹️ End:    Type 'end' to end the trip.\n
            
            ❌ To exit the taximeter, please press 'Escape'.""")
#---------------------------------------------------------------------------------------

def load_rates():
    rates_file_path = os.path.join("intermediate_level", "rates.json")
    with open("rates.json", "r") as file:
        json_data = json.load(file)  
    return json_data["rates"]  

rates = load_rates()
#---------------------------------------------------------------------------------------

def demand_level():
    now = datetime.datetime.now()
    month = now.month
    weekday = now.weekday()
    hour = now.hour

    month_high_demand = [6,7,9,12]
    month_low_demand = [2,3,4,11]
    
    hour_high_demand_week = list(range(7, 10)) + list(range(18, 22))
    hour_high_demand_saturday = list(range(1, 4)) + list(range(22, 24)) 
    hour_high_demand_sunday = list(range(18,21))

    if weekday < 5:
        high_demand_hour = hour_high_demand_week
    elif weekday == 5:
        high_demand_hour = hour_high_demand_saturday
    else:
        high_demand_hour = hour_high_demand_sunday 

    if month in month_high_demand and hour in high_demand_hour:
        return "🔴 High Demand: Extra percentage will be applied during rush hours."
    elif month in month_low_demand and hour  not in high_demand_hour:
        return "🟢 Low Demand: Discounted rate for off-peak hours will be applied"
    else: 
        return "🟡 Normal: Standard rate used during regular traffic conditions"
#---------------------------------------------------------------------------------------

def log_ride(total_ride, demand_multiplier):
    log_entry = {
        "total_ride": total_ride,
        "demand_multiplier": demand_multiplier,
        "timestamp": datetime.datetime.now().isoformat()
    }

    log_folder = "logs"
    log_file_path = os.path.join(log_folder, "ride_logs.json")

    os.makedirs(log_folder, exist_ok=True)
    
    with open(log_file_path, "a") as log_file:
        json.dump(log_entry, log_file)
        log_file.write("\n")  # New line for each entry
#---------------------------------------------------------------------------------------

def ride():
    rate_stationary = rates["base"]["stationary"]
    rate_motion = rates["base"]["motion"]
    
    while True:
        input("Press ENTER to start a new ride 🚖...")
        print("""Let's go! 💨 
              Type the below accordingly:
              ⏸️ Stop
              ⏯️ Go
              ⏹️ End""")
        
        total_stationary = 0
        total_motion = 0
        total_ride = 0
        start_taximeter = time.time()

        while True:
            input_rider = input("Enter: ⏸️ Stop, ⏯️ Go or ⏹️ End: ").strip().lower()
            elapsed_time = time.time() - start_taximeter
            
            if input_rider == "stop":
                total_stationary += elapsed_time * rate_stationary
                print(f"Taxi stopped. Rate: {total_stationary:.2f}€")
                start_taximeter = time.time()
            
            elif input_rider == "go":
                total_motion += (time.time() - start_taximeter) * rate_motion
                print(f"Taxi in motion. Rate: {total_motion:.2f}€")
                start_taximeter = time.time()
            
            elif input_rider == "end":
                total_ride = total_motion + total_stationary
                demand_multiplier = rates["demand_multipliers"]["normal"]  # Default multiplier
                demand_status = demand_level()
                
                if demand_status == "high":
                    demand_multiplier = rates["demand_multipliers"]["high"]
                elif demand_status == "low":
                    demand_multiplier = rates["demand_multipliers"]["low"]
                
                total_ride *= demand_multiplier
                print(f"\nRide ended. TOTAL RATE: {total_ride:.2f}€ (after demand adjustment)\n")
                
                log_ride(total_ride, demand_multiplier)
                break
            
            else:
                print("Invalid input. Enter: ⏸️ Stop, ⏯️ Go or ⏹️ End: ")
        
        print("Do you want to enter a new ride? ✅ Yes ❌ No : ")
        input_restart = input("Type '✅ Yes' if you want a new ride: ").strip().lower()
        if input_restart != 'yes':
            print("Thank you for using Taximeter! Hope to see you again 👋!")
            break
        else:
            print(f"New ride in progress 🚖")
            continue
#---------------------------------------------------------------------------------------


if __name__ == "__main__":
    show_welcome()  # Show the welcome message and instructions
    rates = load_rates()  # Load the rates and store them in a variable
    ride()  # Start the ride process
   



--- Welcome to your taximeter!  🚕 ---
This program calculates the fare in km of a ride in euros.
            
 let me show you how we work first 😉

            The rates used are as follows:

            ⏸️ Taxi stopped: 2 cents per second.
            ▶️ Taxi in motion: 5 cents per second.
            ⬆️ High demand level: extra 10%.
            ⬇️ Low demand level: minus 10%.

Commands are:


            Next when you are on a ride you must use the follow accordingly:


            ▶️ Start:  Press 'Enter' to begin a trip.

            ⏸️ Stop:   Type 'stop' when the taxi is stationary.

            ⏯️ Go:     Type 'go' when the taxi is in motion.

            ⏹️ End:    Type 'end' to end the trip.

            
            ❌ To exit the taximeter, please press 'Escape'.
Let's go! 💨 
              Type the below accordingly:
              ⏸️ Stop
              ⏯️ Go
              ⏹️ End
Taxi in motion. Rate: 0.13€

Ride ended. TOTAL RATE: 0.13€ (after demand adjustment)

Do you want